## Paired Switching

The idea behind this strategy is that if the assets are negatively correlated, then a traditional mixed portfolio might lead to a lower return than the return for the individual assets.

We periodically compute the Co-relation Coefficients' Matrix and choose pair of stocks which have the most negative co-relations and trade in them. The basic idea being that if one of them would fall, then the other would rise and hence, we must switch between them!

We need a re-balancing period ,i.e. The time after which we would be again calculating the co-relations and taking a different pair of stocks : **T**

We would also need to keep switching between these selected stocks in a pair at a certain interval of time : **P**

Next, We need to specify how many such pairs would we be trading in : **M**

The Fraction of the total balance we would be investing on every turn : **R**

The Brokerage Fee as a fraction of the total transaction amount : **F**



In [29]:
import numpy as np
import pandas as pd
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

T = 60
P = 7
M = 4
R = 0.8
F = 0.0005
N=50

### Few Upcoming Functions(GetData and PartitionData) would be similar to the ones you encountered in the Momentum Assignment (Feel free to Copy-Paste your code from there!)

In the file DATA.csv , which we had uploaded in our repository, we have prices of 30 firms enlisted in S & P 500 Index (Apple, IBM, Cisco, Walmart and the like!) from 2nd January 2009 to 17th August 2020.

For our purposes, We'll only be requiring certain columns. On an honest note, Just getting the Columns on Ticker, Date and Adjusted Closing Price would do the job, but if you want, you may take Opening Price as well.

Read up about the [pandas.read_csv](https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html) function from here, and figure out how you'll use it to do the job (You don't need all the columns!) 

In [19]:
def GetData(NameOfFile):
  #START CODE HERE
    a=pd.read_csv(NameOfFile)
    b=a[['datadate','tic','adjcp']]



    return b  # pd.DataFrame Object

To aid Data-Manipulation, it would be beneficial, if we split the DataFrame into many small parts each corresponding to the data corresponding to the 30 Tickers on a particular date. These small parts could then be stored in a list.

We would also be needing to remember which date is at what index, so that we can use that later. 

In [20]:
def PartitionData(Data):
  
    DateToIndex = {}
    
    list=[]
    list1=Data.datadate.unique()
    n=0
    for i in list1:
        list.append(Data[Data.datadate== i])
        DateToIndex[i]=n
        n=n+1





    return list, DateToIndex    # List containing of the Data Partitioned according to Date, and the Dictionary mapping Dates to their index in the list 

Now, We need a function which takes as input a pair of stocks (as indices in range 0-29), Today's date,  alongwith the Partitioned DataFrames which will tell us which of the two stocks has a better outlook for the future, and we'll simply move all our Money from one of them to the other. 

This will be done once in every **P** trading sessions.

You could choose between the two stocks in a number of ways, the simplest of which can be what you did in the last assignment, the Momentum Indicator (i.e. Take the difference between the prices of the stocks in a certain number of days and do your deed! Remember this number of days **N** must be <= **T**)

You may also use other indicators like [RSI](https://www.investopedia.com/terms/r/rsi.asp), [Stochs](https://www.investopedia.com/terms/s/stochasticoscillator.asp), [MACD](https://www.investopedia.com/terms/m/macd.asp) or anything else! (If you decide to use MACD or RSI, you could either implement other functions to calculate them or simply change your ParitionData() function to get the MACD and RSI values as they are already stored there for you!)

You need not worry about getting the nearest dates (or dates at all) in this case, because all you care about is co-relations in long term and growth in short-term so it doesn't matter if you're ahead or behind a few days!

'today' would be an index(integer), i.e. an index on the PartitionedDataFrames list denoting where we are at currently!

Also, as you may have figured it out yourself, while DataFrames are great for Data Handling and small statistical calculations, They aren't so for big calculations as the Numpy Library has both a richer variety of functions for such manipulations and is also more efficient!


In [36]:
def Switch(firstStock, SecondStock, today ,PartitionedDataFrames):

  
    
    
    
    cptoday=np.array(PartitionedDataFrames[today].adjcp)
    cpndayago=np.array(PartitionedDataFrames[today-N].adjcp)
    
    momentum = cptoday-cpndayago    
    
    lcp=[]
    i=today-N
    while i<=today:
        lcp.append(np.array(PartitionedDataFrames[i].adjcp))
        i+=1
    lcp=np.array(lcp)
    meancp = np.mean(lcp,axis=0)
    
    parameter=momentum/meancp
    
    if parameter[firstStock]>parameter[SecondStock]:
        index=firstStock
    else:
        index=SecondStock
    




    return index     #One of two(firstStock/SecondStock) depending on which should grow more in the future.

Now, We need something to simulate our [Portfolio](https://www.investopedia.com/terms/p/portfolio.asp). In order to do that we need a class, which has certain  basic features and functionalities.

(Again some of the functions here, would be same as the previous assignment, so feel free to Copy-Paste)

Features : 


1.   Your Initial Balance
2.   Your Current Balance (Money which isn't invested!)
3.   A list(/any other container) storing the number of stocks of each ticker currently in possession. (Numpy Array prefered)
4.   Most recent prices of all the stocks (As a Numpy array)
5.   A list of **M** or less(as it might be that less than **M** stocks are negatively co-related) Tuples of indices (i,j) where i and j are the indices (from 0-29) of stocks which are negatively co-related.

Functionalities: 



1.   Calculating current Net Worth (Balance+Total Evaluation of all Stocks owned!) 
2.   Buying a Particular Stock (Keep the Transaction fee in mind!)
3.   Selling a particular Stock whole (Keep the Transaction Fee in mind!)
4.   Change Pairs (**T** period intervals) (Takes PartitionedDataFrame, DateT and today's date as input)
5.   Rebalance Portfolio (**P** period intervals) (Takes O)
6.   Function to change the value of most recent prices stored (Takes Numpy array as input)


You may want to know about Numpy's Corrcoef ([np.corrcoef](https://numpy.org/doc/stable/reference/generated/numpy.corrcoef.html)) function!


In [70]:
class PortFolio:
    def __init__(self,initialbal,slist,prices,plist): 
        #Initialize all variables
        self.initbal=initialbal
        self.currbal=initialbal
        self.stocklist=slist
        self.recentprices=prices
        self.pairlist=plist
        

    def SellStock(self, index):
        #index : The index of the Stock to sell (0-29)
        self.currbal=self.currbal+ (self.stocklist[index]*self.recentprices[index])*(1-F)
        self.stocklist[index]=0
        
  
    def BuyStock(self,index, number):
        #index : The index of the Stock to buy (0-29)
        #number : Number of shares to buy (float)
        self.currbal=self.currbal-(number*self.recentprices[index])*(1+F)
        self.stocklist[index]+=number
        
    
    def CalculateNetWorth(self):
    #Return Net Worth (All Shares' costs+ Balance)
        return self.currbal + np.sum(self.recentprices*np.array(self.stocklist))



    def ChangePricesTo(self,newPriceVector):
    # newPriceVector : Numpy array containing the prices of all the stocks for the current day
        self.recentprices = newPriceVector

    def ChangePairs(self,today,PartitionedDataFrames):  
    # Calls the Switch function for all the pairs of stocks owned
        list1=[]
        for j in self.pairlist:
            list1.append(Switch(j[0],j[1], today ,PartitionedDataFrames))
        
  
    def RebalancePortfolio(self,today,PartitionedDataFrames):
    # Calculates the Co-relation Matrix and changes the list of pairs, which you'll be using to trade
    # You'll have to take care of cases, where a single stock would be involved in two or more of the most negatively co-related Stocks
    # It would be a good idea, to call BuyStock and SellStock from here itself, instead of calling ChangePairs later as it will cause havoc that way
        
        pricearr=[]
        for i in range(today-N,today+1):
            pricearr.append(np.array(PartitionedDataFrames[i].adjcp))
        pricearr=np.array(pricearr)
        pricearr=pricearr.transpose()
        cormatrix=np.corrcoef(pricearr)
        print(cormatrix)
        print(pricearr)
        
    





With that the difficult part is over!

Now, all you need to work on is a main loop, which calls all these functions

In [71]:
myPortfolio = PortFolio(15000,[ 0.        ,  0.        ,  5.0        ,  2.11482043,  0.        , 4.14834453,  0.        ,  0.        ,  0.        ,  0.        , 0.        ,  0.        ,  0.        ,  0.        ,  4.71780549, 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,10.20799204,  0.        ,  0.        ,  0.        ,  0.        ,  0.        ,  3.69389992,  0.        ,  0.        ,  0.        ],[ 47.86571429,  46.28      ,  73.47      , 109.82      , 17.65      , 109.66      , 115.57155716,  41.76      , 160.96      ,  37.46      , 164.05      ,  20.02      ,  59.46      ,  46.84      ,  33.635     ,  76.04      ,  93.22      ,  33.67      ,  26.07      ,  19.3475    ,  20.46      ,  61.9       ,  84.81      ,  59.58      ,  44.38      ,  19.18      ,  37.72      ,  41.01      ,  52.54      ,  85.95      ],[(1,2),(3,4),(5,6),(10,12)])
NetWorthAfterEachTrade = []


#First Get the Data
Data = GetData('DATA.csv')
PartitionedData, DateToIndex= PartitionData(Data)

#

pcount,tcount=0,0
#Start processing from the (T+1)th Day(among the ones recorded in the Data)
for i in range(T,2925):
  # Change the Prices to the ith Term
  # Get NetWorth and store in list
  # Check if you need to rebalance Portfolio's Today
  # Check if you need to switch stocks today
    newPriceVector=np.array((PartitionedData[i].adjcp))
    myPortfolio.ChangePricesTo(newPriceVector)
    NetWorthAfterEachTrade.append(myPortfolio.CalculateNetWorth())
    
    tcount+=1
    pcount+=1
    if tcount==T:
        myPortfolio.RebalancePortfolio(i,PartitionedData)
        
    if pcount==P:
        
        myPortfolio.ChangePairs(i,PartitionedData)
    



[1, 4, 5, 10]
[[ 1.          0.52970686  0.8731185   0.4726736   0.7821628   0.61453669
   0.64702705  0.7043092   0.85069889 -0.47695799  0.86835069  0.68196941
   0.66547817  0.37817449  0.75485071  0.76608452  0.7344113   0.42166696
   0.76154523  0.81878349  0.34318901  0.50703415  0.86455577  0.46923204
   0.19546942  0.60086333 -0.5592036   0.52010978  0.06512656  0.7598811 ]
 [ 0.52970686  1.          0.62948053  0.82912277  0.73565973  0.55742031
   0.74975964  0.76722063  0.66526447 -0.06840482  0.54529333  0.32181595
   0.51336355  0.78053738  0.16398985  0.10505947  0.86901741  0.13837007
   0.31710151  0.30454204  0.34328244  0.39010378  0.70183464 -0.07708379
   0.42620578  0.80798415 -0.51906897  0.61324158  0.09465395  0.56901741]
 [ 0.8731185   0.62948053  1.          0.63055937  0.77183091  0.76744418
   0.79306098  0.84765414  0.90811561 -0.52376241  0.85170109  0.53511011
   0.81376962  0.4879298   0.77965261  0.68581447  0.86569735  0.48432906
   0.73453642  0.61167

You may need to declare the PartitionedData list global, depending on your implementation of the Above snippet as well as the various functions in the Portfolio class.

##Moment of Truth

Time to check, if your Program actually works!

Plot the data you collected in various ways (Profits, PercentageProfits etc.) and see if what you did worked!

Feel free to use whichever one of Matplotlib or Seaborn you want to.

Feel free to copy-paste your code from the previous assignment!

In [ ]:
def VizualizeData(NetWorthAfterEachTrade):


        plt.plot()





  

You may use this cell to write about your results/performance of the model!

In [22]:
Data=GetData('DATA.csv')

In [23]:
a,b=PartitionData(Data)

In [31]:
a[571],a[521]

(       datadate   tic       adjcp
 17130  20110408  AAPL   47.865714
 17131  20110408   AXP   46.280000
 17132  20110408    BA   73.470000
 17133  20110408   CAT  109.820000
 17134  20110408  CSCO   17.650000
 17135  20110408   CVX  109.660000
 17136  20110408    DD  115.571557
 17137  20110408   DIS   41.760000
 17138  20110408    GS  160.960000
 17139  20110408    HD   37.460000
 17140  20110408   IBM  164.050000
 17141  20110408  INTC   20.020000
 17142  20110408   JNJ   59.460000
 17143  20110408   JPM   46.840000
 17144  20110408    KO   33.635000
 17145  20110408   MCD   76.040000
 17146  20110408   MMM   93.220000
 17147  20110408   MRK   33.670000
 17148  20110408  MSFT   26.070000
 17149  20110408   NKE   19.347500
 17150  20110408   PFE   20.460000
 17151  20110408    PG   61.900000
 17152  20110408   RTX   84.810000
 17153  20110408   TRV   59.580000
 17154  20110408   UNH   44.380000
 17155  20110408     V   19.180000
 17156  20110408    VZ   37.720000
 17157  20110408   W

In [44]:
np.array(a[571].adjcp)

array([ 47.86571429,  46.28      ,  73.47      , 109.82      ,
        17.65      , 109.66      , 115.57155716,  41.76      ,
       160.96      ,  37.46      , 164.05      ,  20.02      ,
        59.46      ,  46.84      ,  33.635     ,  76.04      ,
        93.22      ,  33.67      ,  26.07      ,  19.3475    ,
        20.46      ,  61.9       ,  84.81      ,  59.58      ,
        44.38      ,  19.18      ,  37.72      ,  41.01      ,
        52.54      ,  85.95      ])

In [37]:
Switch(11,25,571,a)

25

In [41]:
p=np.array([1,2,3,4,5,6])
np.corrcoef?